# Importing Required Libraries



In [19]:
import spacy                                            #for NLP processing
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter                         #for counting word frequencies
from heapq import nlargest                              # For selecting top sentences

import tkinter as tk                                    #For GUI
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import END

# Summarizing Text

In [20]:
def summarize_text():
    # Get text from the text box
    text = text_box.get("1.0", "end-1c")
    
    # Loading spaCy model
    nlp = spacy.load('en_core_web_sm')     #loads a pre-trained English NLP model
    
    # Tokenization and removing stopwords
    doc = nlp(text)                        #performs tokenization,NER,POS Tagging, and dependency parsing
    tokens = [token.text.lower() for token in doc 
              if not token.is_stop and
                not token.is_punct and
                  token.text != '\n']

    # Calculating word frequency
    word_freq = Counter(tokens)            #creates a dictionary where words are keys, and their frequency is the value
    if not word_freq:
        messagebox.showerror("Warning", "Text contains only stopwords or punctuation. Please enter meaningful content.")
        return
    
    # Normalization (so that values are b/w 0 and 1)
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = word_freq[word]/max_freq
      
    # Sentence tokenization
    sent_token = [sent.text for sent in doc.sents]

    sent_score = {}                       #dictionary where Keys = Sentences, Values = Sentence scores
    for sent in sent_token:
        for word in sent.split():
            if word.lower() in word_freq.keys():
                if sent not in sent_score.keys():
                    sent_score[sent] = word_freq[word]
                else:
                    sent_score[sent] += word_freq[word]

    # Select top-scoring sentences based on user input
    num_sentences = int(num_sentences_entry.get())                                                       #no of sentences user wants
    summarized_sentences = nlargest(num_sentences, sent_score, key=sent_score.get)

    # Display summarized text in the result box
    result_box.delete(1.0, END)
    result_box.insert(END, " ".join(summarized_sentences))



# GUI setup

In [21]:
root = tk.Tk()
root.title("Text Summarizer")

# Text box for input
text_box = scrolledtext.ScrolledText(root, width=100, height=15, wrap=tk.WORD)
text_box.pack(pady=10)

# Entry field for the number of sentences
num_sentences_label = tk.Label(root, text="Number of Sentences:")
num_sentences_label.pack()
num_sentences_entry = tk.Entry(root, width=10)
num_sentences_entry.insert(END, "3")  # Default value
num_sentences_entry.pack()

# Button to summarize
summarize_button = tk.Button(root, text="Summarize", command=summarize_text)
summarize_button.pack(pady=5)

# Result box for output
result_box = scrolledtext.ScrolledText(root, width=100, height=15, wrap=tk.WORD)
result_box.pack(pady=10)

root.mainloop()
